In [2]:
import numpy as np
import pandas as pd
import sys
import os
import shutil

<h2>데이터 전처리<h2>

In [61]:
def read_text_files_to_dataframe(folder_path):
    
    latitude = []
    longitude = []
    date = []
    time = []

    # 폴더 내의 모든 텍스트 파일을 읽음
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            
            # 텍스트 파일을 DataFrame에 추가

            with open(file_path, 'r') as file:
                data = file.read()

            data = data.split('\n')[6:-1]
            for i in range(len(data)):
                line = data[i].split(',')
                latitude.append(line[0])
                longitude.append(line[1])
                date.append(line[5])
                time.append(line[6])

    df = pd.DataFrame({"latitude":latitude, "longitude":longitude, "date":date, "time":time})
    df['latitude'] = df['latitude'].astype(float)
    df['longitude'] = df['longitude'].astype(float)
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y-%m-%d %H:%M:%S')
    df['datetime'] = df['datetime'].dt.floor('T')
    df = df.drop(['date', 'time'], axis=1)
    df = df.drop_duplicates(['datetime'], ignore_index=True)

    return df

In [62]:
folder_path = "C:\\Users\\sk002\\OneDrive\\바탕 화면\\학교\\Yoodori\\Geolife Trajectories 1.3\\Data\\000\\Trajectory"

dataframe = read_text_files_to_dataframe(folder_path)

dataframe

,latitude,longitude,datetime
0,39.984702,116.318417,2008-10-23 02:53:00
1,39.984586,116.316716,2008-10-23 02:54:00
2,39.984485,116.314426,2008-10-23 02:55:00
3,39.984696,116.312921,2008-10-23 02:56:00
4,39.984656,116.310660,2008-10-23 02:57:00
...,...,...,...
15118,39.996230,116.326576,2009-07-05 07:41:00
15119,39.999069,116.328971,2009-07-05 07:42:00
15120,39.999975,116.327367,2009-07-05 07:43:00
15121,39.999974,116.327326,2009-07-05 07:44:00


<h2>의미장소 추출<h2>

In [2]:
from pyclustering.cluster.gmeans import gmeans
from pyclustering.cluster import cluster_visualizer
from pyclustering.cluster import cluster_visualizer_multidim
import folium

In [1]:
def gmeans(df):
    # 두 열을 선택하고 넘파이 배열로 변환
    selected_columns = ['latitude', 'longitude']
    result_list = df[selected_columns].values.tolist()
    
    gmeans_instance = gmeans(result_list).process()

    # Extract clustering results: clusters and their centers
    clusters = gmeans_instance.get_clusters()
    centers = gmeans_instance.get_centers()

    return clusters, centers
    

In [3]:
def map(centers, i):

    # 예제 데이터 생성 (서울의 위도, 경도)
    locations = centers

    # 지도의 중심 좌표 설정
    map_center = [sum(x[0] for x in locations) / len(locations), sum(x[1] for x in locations) / len(locations)]

    # Folium 맵 생성
    mymap = folium.Map(location=map_center, zoom_start=12)

    # 위치 데이터를 지도에 마커로 표시
    for loc in locations:
        folium.Marker(location=loc, popup='Location').add_to(mymap)

    # 지도를 HTML 파일로 저장하거나 주피터 노트북에서 바로 표시
    file_name = f'map{i}.html'
    mymap.save(file_name)


In [4]:
def app(folder_path):
    
    j = 0
    data_dict = {}
    # 폴더 내의 모든 텍스트 파일을 읽음
    for filename in os.listdir(folder_path):
        latitude = []
        longitude = []
        date = []
        time = []
        file_path = os.path.join(folder_path, filename)
            
        # 텍스트 파일을 DataFrame에 추가

        with open(file_path, 'r') as file: 
            data = file.read()

        data = data.split('\n')[6:-1]
        for i in range(len(data)):
            line = data[i].split(',')
            latitude.append(line[0])
            longitude.append(line[1])
            date.append(line[5])
            time.append(line[6])
        
        df = pd.DataFrame({"latitude":latitude, "longitude":longitude, "date":date, "time":time})
        df['latitude'] = df['latitude'].astype(float)
        df['longitude'] = df['longitude'].astype(float)
        df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y-%m-%d %H:%M:%S')
        df['datetime'] = df['datetime'].dt.floor('T')
        df = df.drop(['date', 'time'], axis=1)
        df = df.drop_duplicates(['datetime'], ignore_index=True)
        clusters, centers = gmeans(df)
        data_dict[j] = pd.DataFrame({"clusters":clusters, "centers":centers})
        
        map(centers, j)
        
        j += 1
    return data_dict

In [ ]:
# 
# folder_name = r'C:\Users\sk002\OneDrive\바탕 화면\학교\Yoodori\Geolife Trajectories 1.3\Data\000\Trajectory'
# data_dict = app(folder_name)